In [ ]:
import sys
import os

sys.path.insert(0, os.path.abspath("."))
sys.path.append(os.path.abspath("../../../"))

In [ ]:
from desc import set_device
set_device("gpu")

In [ ]:
import numpy as np
np.set_printoptions(linewidth=np.inf, precision=4, suppress=True, threshold=sys.maxsize)
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objects as go
import functools
import scipy

In [ ]:
import desc

from desc.basis import *
from desc.backend import *
from desc.compute import *
from desc.coils import *
from desc.equilibrium import *
from desc.examples import *
from desc.grid import *
from desc.geometry import *

from desc.objectives import *
from desc.objectives.objective_funs import *
from desc.objectives.getters import *
from desc.objectives.normalization import compute_scaling_factors
from desc.objectives.utils import *
from desc.optimize._constraint_wrappers import *

from desc.transform import Transform
from desc.plotting import *
from desc.optimize import *
from desc.perturbations import *
from desc.profiles import *
from desc.compat import *
from desc.utils import *
from desc.magnetic_fields import *

from desc.__main__ import main
from desc.vmec_utils import vmec_boundary_subspace
from desc.input_reader import InputReader
from desc.continuation import solve_continuation_automatic

print_backend_info()

In [ ]:
def plot_field_lines(field, equ, ntransit=200, nrho=9):
    # for starting locations we'll pick positions on flux surfaces on the outboard midplane
    grid_trace = LinearGrid(rho=np.linspace(0, 1, nrho))
    r0 = equ.compute("R", grid=grid_trace)["R"]
    z0 = equ.compute("Z", grid=grid_trace)["Z"]
    fig, ax = desc.plotting.plot_surfaces(equ)
    fig, ax = desc.plotting.poincare_plot(
        field,
        r0,
        z0,
        NFP=equ.NFP,
        ax=ax,
        color="k",
        size=0.5,
        ntransit=ntransit,
    )
    return fig, ax

def plot_init_coils_modular(equ, ncoils=3, r_over_a=3.5):
    coilset = initialize_modular_coils(
        equ, num_coils=ncoils, r_over_a=r_over_a
    ).to_FourierXYZ()
    fig = plot_3d(equ, "|B|")
    fig = plot_coils(coilset, fig=fig)
    return fig

In [ ]:
def optimize_coils(equ, coilset0=None):
    if coilset0 is None:
        coilset0 = initialize_modular_coils(
            equ, num_coils=3, r_over_a=3.5
        ).to_FourierXYZ()
    # define our objective function (we will use a helper function here to make it easier to change weights later)
    weights = {
        "quadratic flux": 500,
        "coil-coil min dist": 100,
        "plasma-coil min dist": 10,
        "coil curvature": 250,
        "coil length": 20,
    }
    coil_grid = LinearGrid(N=50)
    # similarly define a grid on the plasma surface where B*n errors will be evaluated
    plasma_grid = LinearGrid(M=25, N=25, NFP=equ.NFP, sym=equ.sym)
    obj = ObjectiveFunction(
        (
            QuadraticFlux(
                equ,
                field=coilset0,
                # grid of points on plasma surface to evaluate normal field error
                eval_grid=plasma_grid,
                field_grid=coil_grid,
                vacuum=True,  # vacuum=True means we won't calculate the plasma contribution to B as it is zero
                weight=weights["quadratic flux"],
                bs_chunk_size=10,
            ),
            CoilCurvature(
                coilset0,
                bounds=(-1, 2),
                normalize_target=False,  # we're giving bounds in normalized units
                grid=coil_grid,
                weight=weights["coil curvature"],
            ),
            CoilLength(
                coilset0,
                bounds=(0, 2 * np.pi * (coilset0[0].compute("length")["length"])),
                normalize_target=True,  # target length is in meters, not normalized
                grid=coil_grid,
                weight=weights["coil length"],
            ),
        )
    )
    coil_indices_to_fix_current = [False for c in coilset0]
    coil_indices_to_fix_current[0] = True
    constraints = (FixCoilCurrent(coilset0, indices=coil_indices_to_fix_current),)

    optimizer = Optimizer("lsq-exact")

    (optimized_coilset,), _ = optimizer.optimize(
        coilset0,
        objective=obj,
        constraints=constraints,
        maxiter=200,
        verbose=3,
        ftol=1e-6,
        xtol=0,
        copy=True,
    )
    return optimized_coilset

In [ ]:
def optimize_coils_regcoil(surf, equ, num_coils=16):
    # create the FourierCurrentPotentialField object from the constant offset surface we found in the previous cell
    surface_current_field = FourierCurrentPotentialField.from_surface(
        surf,
        I=0,
        # manually setting G to value needed to provide the equilibrium's toroidal flux,
        # though this is not necessary as it gets set automatically inside the solve_regularized_surface_current function
        G=np.asarray(
            [
                -equ.compute("G", grid=LinearGrid(rho=np.array(1.0)))["G"][0]
                / mu_0
                * 2
                * np.pi
            ]
        ),
        # set symmetry of the current potential, "sin" is usually expected for stellarator-symmetric surfaces and equilibria
        sym_Phi="sin",
    )

    surface_current_field.change_Phi_resolution(M=12, N=12)

    # create the evaluation grid (where Bn will be minimized on plasma surface)
    # and source grid (discretizes the source K for Biot-Savart and where |K| will be penalized on winding surface)
    Megrid = 20
    Negrid = 20
    Msgrid = 20
    Nsgrid = 20

    eval_grid = LinearGrid(M=Megrid, N=Negrid, NFP=eq.NFP, sym=False)
    # ensure that sym=False for source grid so the field evaluated from the surface current is accurate
    # (i.e. must evaluate source over whole surface, not just the symmetric part)
    # NFP>1 is ok, as we internally will rotate the source through the field periods to sample entire winding surface
    sgrid = LinearGrid(M=Msgrid, N=Nsgrid, NFP=eq.NFP, sym=False)

    lambda_regularization = np.append(np.array([0]), np.logspace(-30, 1, 20))

    # solve_regularized_surface_current method runs the REGCOIL algorithm
    fields, data = solve_regularized_surface_current(
        surface_current_field,  # the surface current field whose geometry and Phi resolution will be used
        eq=equ,  # the Equilibrium object to minimize Bn on the surface of
        source_grid=sgrid,  # source grid
        eval_grid=eval_grid,  # evaluation grid
        current_helicity=(
            1,
            0,
        ),  # pair of integers (M_coil, N_coil), determines topology of contours (almost like  QS helicity),
        #  M_coil is the number of times the coil transits poloidally before closing back on itself
        # and N_coil is the toroidal analog (if M_coil!=0 and N_coil=0, we have modular coils, if both M_coil
        # and N_coil are nonzero, we have helical coils)
        # we pass in an array to perform scan over the regularization parameter (which we call lambda_regularization)
        # to see tradeoff between Bn and current complexity
        lambda_regularization=lambda_regularization,
        # lambda_regularization can also be just a single number in which case no scan is performed
        vacuum=True,  # this is a vacuum equilibrium, so no need to calculate the Bn contribution from the plasma currents
        regularization_type="regcoil",
        chunk_size=40,
    )
    surface_current_field = fields[
        0
    ]  # fields is a list of FourierCurrentPotentialField objects

    coilset = surface_current_field.to_CoilSet(num_coils=num_coils, stell_sym=True)
    
    return coilset

In [ ]:
def solve_n0_fixed(eq2solve, **kwargs):
    jac_chunk_size = kwargs.pop("jac_chunk_size", None)
    R_modes = eq2solve.R_basis.modes[eq2solve.R_basis.modes[:, 2] == 0]
    Z_modes = eq2solve.Z_basis.modes[eq2solve.Z_basis.modes[:, 2] == 0]
    cons = (
        FixModeR(eq2solve, modes=R_modes),
        FixModeZ(eq2solve, modes=Z_modes),
        FixPressure(eq2solve),
        FixPsi(eq2solve),
        FixCurrent(eq2solve),
        FixSheetCurrent(eq2solve),
        FixLambdaGauge(eq2solve),
    )
    cons = maybe_add_self_consistency(eq2solve, cons)
    obj = ObjectiveFunction(ForceBalance(eq2solve, jac_chunk_size=jac_chunk_size))
    eq2solve.solve(
        constraints=cons,
        objective=obj,
        **kwargs,
    )

# Now solve with Fixed N=0 Modes

In [ ]:
eq_poin = eq.copy()
solve_n0_fixed(eq_poin, maxiter=500, ftol=5e-4)

In [ ]:
eq_poin.save(f"landreman2021-island-poincare-solved-L{eq_poin.L}M{eq_poin.M}N{eq_poin.N}.h5")

As seen below, the boundary change is minimal. If we haven't solved for high res, this wouldn't be the case.

In [ ]:
plot_comparison(eqs=[eq, eq_poin], labels=["original", "resolve poincare"]);

Optimize some coils for field line tracing.

In [ ]:
# create the constant offset surface
surf2 = eq_poin.surface.constant_offset_surface(
    offset=0.25,  # desired offset
    M=16,  # Poloidal resolution of desired offset surface
    N=16,  # Toroidal resolution of desired offset surface
    grid=LinearGrid(M=32, N=32, NFP=eq_poin.NFP),
)  # grid of points on base surface to evaluate unit normal and find points on offset surface,
# generally should be twice the desired resolution
optimized_coilset2 = optimize_coils_regcoil(surf2, eq_poin)

In [ ]:
eq_poin.surface = eq_poin.get_surface_at(rho=1.)
fig = plot_3d(
    eq_poin.surface,
    "B*n",
    field=optimized_coilset2,
    field_grid=coil_grid,
    grid=plot_grid,
)

fig = plot_coils(optimized_coilset2, fig=fig)
fig.show()

In [ ]:
fig, ax = plot_field_lines(optimized_coilset2, eq_poin, nrho=18, ntransit=200)
plt.show()
# fig.savefig("landreman2021-islands-after-poincare.png", dpi=1000)

In [ ]:
plot_boozer_surface(eq_poin)
plt.title("After Poincare")
plot_boozer_surface(eq)
plt.title("Original");

In [ ]:
levels = np.logspace(-1,5, 7)
plot_section(eq_poin, "|F|", phi=3, log=True, levels=levels);
plt.title("After Poincare")
plot_section(eq, "|F|", phi=3, log=True, levels=levels);
plt.title("Original");

In [ ]:
plot_1d(eq, "iota")
plot_1d(eq_poin, "iota");

In [ ]:
plot_1d(eq, "current")
plot_1d(eq_poin, "current");

# Don't fix $\lambda$

In [ ]:
eq._xsection = eq.get_surface_at(zeta=0)
eq_poin_noL = eq.copy()
solve_poincare(eq_poin_noL, maxiter=350, ftol=5e-4, fix_lambda=False)
eq_poin_noL.surface = eq_poin_noL.get_surface_at(rho=1)

In [ ]:
plot_comparison([eq, eq_poin_noL], labels=["original", "poincare no L"]);

In [ ]:
# create the constant offset surface
surf3 = eq_poin_noL.surface.constant_offset_surface(
    offset=0.25,  # desired offset
    M=16,  # Poloidal resolution of desired offset surface
    N=16,  # Toroidal resolution of desired offset surface
    grid=LinearGrid(M=32, N=32, NFP=eq_poin_noL.NFP),
)  # grid of points on base surface to evaluate unit normal and find points on offset surface,
# generally should be twice the desired resolution
optimized_coilset3 = optimize_coils_regcoil(surf3, eq_poin_noL)

In [ ]:
eq_poin_noL.surface = eq_poin_noL.get_surface_at(rho=1.0)
fig = plot_3d(
    eq_poin_noL.surface,
    "B*n",
    field=optimized_coilset3,
    field_grid=coil_grid,
    grid=plot_grid,
)

fig = plot_coils(optimized_coilset3, fig=fig)
fig.show()

In [ ]:
fig, ax = plot_field_lines(optimized_coilset3, eq_poin_noL, nrho=18, ntransit=200)
plt.show()
# fig.savefig("landreman2021-islands-after-poincare.png", dpi=1000)

In [ ]:
plot_1d(eq_poin_noL, "iota")

# Fix n=0 modes

In [ ]:
eq_poin0 = eq.copy()
R_modes = eq_poin0.R_basis.modes[eq_poin0.R_basis.modes[:, 2] == 0]
Z_modes = eq_poin0.Z_basis.modes[eq_poin0.Z_basis.modes[:, 2] == 0]
cons = (
    FixModeR(eq_poin0, modes=R_modes),
    FixModeZ(eq_poin0, modes=Z_modes),
    FixPressure(eq_poin0),
    FixPsi(eq_poin0),
    FixCurrent(eq_poin0),
    FixSheetCurrent(eq_poin0),
    FixLambdaGauge(eq_poin0),
)
cons = maybe_add_self_consistency(eq_poin0, cons)
obj = ObjectiveFunction(ForceBalance(eq_poin0))
eq_poin0.solve(
    constraints=cons,
    objective=obj,
    maxiter=500,
    ftol=5e-4,
    verbose=3,
)

In [ ]:
plot_comparison([eq, eq_poin0], labels=["original", "fix n=0 modes"]);

In [ ]:
# create the constant offset surface
surf4 = eq_poin0.surface.constant_offset_surface(
    offset=0.25,  # desired offset
    M=16,  # Poloidal resolution of desired offset surface
    N=16,  # Toroidal resolution of desired offset surface
    grid=LinearGrid(M=32, N=32, NFP=eq_poin0.NFP),
)  # grid of points on base surface to evaluate unit normal and find points on offset surface,
# generally should be twice the desired resolution
optimized_coilset4 = optimize_coils_regcoil(surf3, eq_poin0)

In [ ]:
eq_poin0.surface = eq_poin0.get_surface_at(rho=1.0)
fig = plot_3d(
    eq_poin0.surface,
    "B*n",
    field=optimized_coilset4,
    field_grid=coil_grid,
    grid=plot_grid,
)

fig = plot_coils(optimized_coilset4, fig=fig)
fig.show()

In [ ]:
fig, ax = plot_field_lines(optimized_coilset4, eq_poin0, nrho=18, ntransit=200)
plt.show()
fig.savefig("landreman2021-islands-after-pfixed-n0-modes.png", dpi=1000)

In [ ]:
plot_1d(eq_poin0, "iota")
plot_1d(eq, "iota")

In [ ]:
levels = np.logspace(-1, 5, 7)
plot_section(eq_poin0, "|F|", phi=3, log=True, levels=levels)
plt.title("After Poincare")
plot_section(eq, "|F|", phi=3, log=True, levels=levels)
plt.title("Original")